Data Model
class in python is runtime executable code.


In [25]:
class Polynomial:
    print("In class Polynomial")
    pass


In class Polynomial


In [26]:
p1 = Polynomial()

In [27]:
p1.coeffs  = 1, 2, 3 # x^2 + 2x +3

In [28]:
p1

In [29]:
p2 = Polynomial()

In [30]:
p2.coeffs = 3, 4, 3

In [31]:
p2

Take it ahead and add a init for coeffs

In [32]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs

In [33]:
p1 = Polynomial(1,2,3)
p2 = Polynomial(3,4,5)

In [34]:
p1, p2 

(<__main__.Polynomial at 0x225dfe6dbe0>,
 <__main__.Polynomial at 0x225dfe6dc88>)



Some method to return the printable representation of the class to the screen.

In [35]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
    
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    

In [36]:
p1 = Polynomial(1,2,3)
p1

Polynomial(*(1, 2, 3))

In [37]:
p2 = Polynomial(3,4,3)
p2

Polynomial(*(3, 4, 3))

If I want to add, we will just add another dunder/double underscore methods or data model methods

1.some behavior that I want to implement --> write some __function__

2.top level function or top-level syntax--> corresponding __
    x +y --> __add__
    init x --> __init__
    repr(x) --> __repr__
    x() --> __call__

In [38]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
    
    def __repr__(self):
        return 'Polynomial(*{!r})'.format(self.coeffs)
    
    def __add__(self,other):
        return Polynomial(*(x + y for x,y in zip(self.coeffs,other.coeffs)))
    
    def __len__(self):
        return len(self.coeffs)
    
    def __call__(self):
        pass

In [39]:
p1 = Polynomial(1,2,3)
p2 = Polynomial(3,4,3)
p1 + p2

Polynomial(*(4, 6, 6))

In [40]:
len(p1)

3

3 core patterns to understand OOP in Python
-the protocol view of python
-built in inheritance protocol and how it works (where to go on a python object to look for things)
-some caveats around how OO in python works.


    Core units works on the basic core functionalities
    
    the user side/ business logoc side works with utilising these core features for business ends (Library code)

# metaclasses


In [41]:
#library.py

class Base:
    def foo(self):
        return 'foo'

In [42]:
#user.py

# from library import Base

class Derived(Base):
    def bar(self):
        return self.foo()

adding things to userside code/  or lib(core) code so that the code fails before hitting runtime . get the failures before run time 

#user side/business code, add a check for the library function

In [43]:
#library.py

class Base:
    def foo(self):
        return 'foo'

In [44]:
#user.py

# from library import Base

assert hasattr(Base,'foo'),"you broke it , you fool!"

class Derived(Base):
    def bar(self):
        return self.foo()




#library/core side code, add a check for the library function



#dis library, to disassemble

In [45]:
def _():
    class Base:
        pass

from dis import dis


In [46]:
dis(_)

  2           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object Base at 0x00000225DFE3A9C0, file "<ipython-input-45-a194b247271c>", line 2>)
              4 LOAD_CONST               2 ('Base')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('Base')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (Base)
             14 LOAD_CONST               0 (None)
             16 RETURN_VALUE


In [48]:
# catch building of class

class Base :
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(*a,**kwargs):
    print('my_buildclass-->',a, kwargs)

import builtins
builtins.__build_class__ = my_bc

In [49]:
# Catch Building of Classes

class Base:
    def foo(self):
        return self.bar()

old_bc = __build_class__
def my_bc(fun, name, base=None, **kw):
    if base is Base:
                print('Check if bar method defined')
    if base is not None:
        return old_bc(fun, name, base, **kw)
    return old_bc(fun, name, **kw)

import builtins
builtins.__build_class__ = my_bc

my_buildclass--> (<function Base at 0x00000225DFE54AE8>, 'Base') {}


In [50]:
#user.py


class Derived(Base):
    def bar(self):
        return 'bar'

Check if bar method defined
my_buildclass--> (<function Derived at 0x00000225DFE54488>, 'Derived') {}


#Build class is a solution to this, but ppl fundamentaly dont use it, instead Metaclass is used.


In [51]:
#library.py
class BaseMeta(type):
    def __new__(cls,name, bases,body):
        print('BaseMeta,__new__',cls,name,bases,body)
        return super().__new__(cls,name,bases,body)

        class Base(metaclass=BaseMeta):
            def foo(self):
                return self.bar()
    

my_buildclass--> (<function BaseMeta at 0x00000225DFE1CEA0>, 'BaseMeta', <class 'type'>) {}


In [52]:
#user.py


class Derived(Base):
    def bar(self):
        return 'bar'

Check if bar method defined
my_buildclass--> (<function Derived at 0x00000225DFE3BE18>, 'Derived') {}


In [53]:
#library.py
# this is library /core code instance.
#updated with raise, as we cant do asserts as in user side code.
class BaseMeta(type):
    def __new__(cls, name, bases, body):
        if name != 'Base' and not 'bar' in body:
            raise TypeError('bad user class')
        return super().__new__(cls, name, bases, body)

class Base(metaclass=BaseMeta):
    def foo(self):
        return self.bar()
    
    def __init_subclass__(*a, **kw):
        print('init_subclass', a, kw)
        return super().__init_subclass__(*a, **kw)

my_buildclass--> (<function BaseMeta at 0x00000225DFE4DD08>, 'BaseMeta', <class 'type'>) {}
Check if bar method defined
my_buildclass--> (<function Base at 0x00000225DFE4DD08>, 'Base') {'metaclass': None}


metaclass implementation is not so easy, its clumsy to use.. 

in python 3.6 a new method was use,,, subclass.

In [54]:
help(Base.__init_subclass__)

Help on built-in function __init_subclass__:

__init_subclass__(...) method of builtins.type instance
    This method is called when a class is subclassed.
    
    The default implementation does nothing. It may be
    overridden to extend subclasses.



# Decorator

In [55]:
#dec.py
def add(x,y=10):
    return x + y

In [56]:
add

<function __main__.add>

In [57]:
add(20,10)

30

In [58]:
add.__name__

'add'

In [59]:
add.__module__

'__main__'

In [60]:
add.__defaults__

(10,)

In [61]:
add.__code__

<code object add at 0x00000225DFCC3E40, file "<ipython-input-55-30c333306bec>", line 2>

In [62]:
add.__code__.co_

AttributeError: 'code' object has no attribute 'co_'

Whats your source code ?


In [63]:
from inspect import getsource

In [64]:
getsource(add)

'def add(x,y=10):\n    return x + y\n'

In [65]:
print(getsource(add))

def add(x,y=10):
    return x + y



In [66]:
#what file you are in .
from inspect import getfile
getfile(add)

'<ipython-input-55-30c333306bec>'

if we want to time the functions , we can use the time module

In [67]:
#dec.py
from time import time

def add(x,y=10):
    return x + y

before = time()
print('add(10)', add(10))
after = time()
print('time taken',after -before)

print('add(20, 30)', add(20, 30))
print('add("a", "b")', add("a", "b"))


add(10) 20
time taken 0.0
add(20, 30) 50
add("a", "b") ab


In [68]:

def add_timer(x, y=10):
    before = time()
    rv = x + y
    after = time()
    print('elapsed:', after - before)
    return rv

In [69]:
print('add(10)', add_timer(10))
print('add(20, 30)', add_timer(20, 30))
print('add("a", "b")', add_timer("a", "b"))

elapsed: 0.0
add(10) 20
elapsed: 0.0
add(20, 30) 50
elapsed: 0.0
add("a", "b") ab


Now adding a time at every other add call is cumbersome and inefficient., hence decorators come in. They are basically wrapping up of a function within another function.

#  But what if we have multiple functions that require timing?


In [70]:
def sub(x,y=10):
    return x - y

In [71]:
print('sub(10)',sub(10))
print('sub(20,30)',sub(20,30))

sub(10) 0
sub(20,30) -10


In [72]:
def timer(func, x, y=10):
    before = time()
    rv = func(x,y)
    after = time()
    print('elapsed',after - before)
    return rv

In [73]:
print('add(10)', timer(add, 10))
print('add(20, 30)', timer(add, 20, 30))
print('add("a", "b")', timer(add, "a", "b"))

elapsed 0.0
add(10) 20
elapsed 0.0
add(20, 30) 50
elapsed 0.0
add("a", "b") ab


make timer return another function

In [74]:
def timer(func):
    def f(x, y=10):
        before = time()
        rv = func(x, y)
        after = time()
        print('elapsed', after - before)
        return rv
    return f

In [75]:
add = timer(add)


In [76]:
print('add(10)', add(10))
print('add(20, 30)', add(20, 30))
print('add("a", "b")', add("a", "b"))

elapsed 0.0
add(10) 20
elapsed 0.0
add(20, 30) 50
elapsed 0.0
add("a", "b") ab


In [77]:
# Don't need to do add = timer(add) with decorators...

In [78]:
@timer
def add_dec(x, y=10):
    return x + y

@timer
def sub_dec(x, y=10):
    return x - y

In [79]:
print('add(10)', add_dec(10))
print('add(20, 30)', add_dec(20, 30))
print('add("a", "b")', add_dec("a", "b"))
print('sub(10)', sub_dec(10))
print('sub(20, 30)', sub_dec(20, 30))

elapsed 0.0
add(10) 20
elapsed 0.0
add(20, 30) 50
elapsed 0.0
add("a", "b") ab
elapsed 0.0
sub(10) 0
elapsed 0.0
sub(20, 30) -10


In [80]:
# we don't need to hardcode parameters in decorator functions
def timer_k(func):
    def f(*args, **kwargs):
        before = time()
        rv = func(*args, **kwargs)
        after = time()
        print('elapsed', after - before)
        return rv
    return f

In [81]:
@timer_k
def add_dec(x, y=10):
    return x + y

@timer_k
def sub_dec(x, y=10):
    return x - y

print('add(10)', add_dec(10))
print('add(20, 30)', add_dec(20, 30))
print('add("a", "b")', add_dec("a", "b"))
print('sub(10)', sub_dec(10))
print('sub(20, 30)', sub_dec(20, 30))

elapsed 0.0
add(10) 20
elapsed 0.0
add(20, 30) 50
elapsed 0.0
add("a", "b") ab
elapsed 0.0
sub(10) 0
elapsed 0.0
sub(20, 30) -10


In [82]:
# What if I want to run a function n number of times
# Let's have add run 3 times in a row and sub run twice in a row

In [83]:

n = 2

def ntimes(f):
    def wrapper(*args, **kwargs):
        for _ in range(n):
            print('running {.__name__}'.format(f))
            rv = f(*args, **kwargs)
        return rv
    return wrapper
    
        
@ntimes
def add_dec(x, y=10):
    return x + y

@ntimes
def sub_dec(x, y=10):
    return x - y

Higher Order Decorators


In [84]:
def ntimes(n):
    def inner(f):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                print('running {.__name__}'.format(f))
                rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner
    
        
@ntimes(2)
def add_hdec(x, y=10):
    return x + y

@ntimes(4)
def sub_hdec(x, y=10):
    return x - y

In [85]:
print('add(10)', add_hdec(10))
print('add(20, 30)', add_hdec(20, 30))
print('add("a", "b")', add_hdec("a", "b"))
print('sub(10)', sub_hdec(10))
print('sub(20, 30)', sub_hdec(20, 30))


running add_hdec
running add_hdec
add(10) 20
running add_hdec
running add_hdec
add(20, 30) 50
running add_hdec
running add_hdec
add("a", "b") ab
running sub_hdec
running sub_hdec
running sub_hdec
running sub_hdec
sub(10) 0
running sub_hdec
running sub_hdec
running sub_hdec
running sub_hdec
sub(20, 30) -10



# Generators

In [1]:
# gen.py - use whenever sequencing is needd
# top-level syntax, function -> underscore method
# x()               __call__

In [3]:
def add1(x, y):
    return x + y


class Adder:
    def __call__(self, x, y):
        return x + y

add2 = Adder()

In [4]:
add1(10, 20)


30

In [5]:
add2(10, 20)


30

In [6]:
def add1(x, y):
    return x + y

class Adder:
    def __init__(self):
        self.z = 0
        
    def __call__(self, x, y):
        self.z += 1
        return x + y + self.z

add2 = Adder()

In [7]:
from time import sleep
# This example has storage... and has eager return of the result sets
def compute():
    rv = []
    for i in range(10):
        sleep(.5)
        rv.append(i)
    return rv

In [8]:
compute()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Wasteful because we have to wait for the entire action to complete and be read into memory, when we really just care about each number (one by one)

In [9]:
class Compute:
    def __call__(self):
        rv = []
        for i in range(100000):
            sleep(5)
            rv.append(i)
        return rv
    
    def __iter__(self):
        self.last = 0
        return self
    
    def __next__(self):
        rv = self.last
        self.last += 1
        if self.last > 10:
            raise StopIteration()
        sleep(.5)
        return self.last
        
compute = Compute()

# THIS IS UGLY... now let's make a generator

In [10]:

#This is a generator... don't eagerly compute. Return to user as they ask for it...

def compute():
    for i in range(10):
        sleep(.5)
        yield i

In [11]:

# for x in xs:
#    pass

# xi = iter(xs)    -> __iter__
# while True:
#   x = next(xi)   -> __next__

In [12]:
for val in compute():
    print(val)

0
1
2
3
4
5
6
7
8
9


In [13]:
#how we run Apis... 

In [14]:
class Api:
    def run_this_first(self):
        first()
    def run_this_second(self):
        second()
    def run_this_last(self):
        last()

In [15]:
#but there is a better way, there is a reason we have been given three different distinct function in the api instead of a single func

In [16]:
#generators can be used as context switchers.
def api():
    first()
    yield
    second()
    yield
    last()

# Context Manager


In [17]:
# cty.py

from sqlite3 import connect

In [19]:
# with ctx() as x:
#   pass

# x = ctx().__enter__
# try:
#   pass
# finally:
#    x.__exit__

class temptable:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        print('__enter__')
        self.cur.execute('create table points(x int, y int)')
    def __exit__(self, *args):
        print('__exit__')
        self.cur.execute('drop table points')

with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

__enter__
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
__exit__


In [20]:
rm test.db

SyntaxError: invalid syntax (<ipython-input-20-eef4ba1d549e>, line 1)

In [21]:
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')
    
class contextmanager:
    def __init__(self, cur):
        self.cur = cur
    def __enter__(self):
        self.gen = temptable(self.cur)
        next(self.gen)
    def __exit__(self, *args):
        next(self.gen, None)
        
with connect('test.db') as conn:
    cur = conn.cursor()
    with contextmanager(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)
        for row in cur.execute('select sum(x * y) from points'):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
(9,)
dropped table


In [22]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)

def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')
temptable = contextmanager(temptable)
            
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
dropped table


In [23]:
class contextmanager:
    def __init__(self, gen):
        self.gen = gen
    def __call__(self, *args, **kwargs):
        self.args, self.kwargs = args, kwargs
        return self
    def __enter__(self):
        self.gen_inst = self.gen(*self.args, **self.kwargs)
        next(self.gen_inst)
    def __exit__(self, *args):
        next(self.gen_inst, None)
        
@contextmanager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    yield
    cur.execute('drop table points')
    print('dropped table')
            
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
dropped table


In [24]:
from sqlite3 import connect
from contextlib import contextmanager
        
@contextmanager
def temptable(cur):
    cur.execute('create table points(x int, y int)')
    print('created table')
    try:
        yield
    finally:
        cur.execute('drop table points')
        print('dropped table')
            
with connect('test.db') as conn:
    cur = conn.cursor()
    with temptable(cur):
        cur.execute('insert into points (x, y) values(1, 1)')
        cur.execute('insert into points (x, y) values(1, 2)')
        cur.execute('insert into points (x, y) values(2, 1)')
        cur.execute('insert into points (x, y) values(2, 2)')
        for row in cur.execute("select x, y from points"):
            print(row)

created table
(1, 1)
(1, 2)
(2, 1)
(2, 2)
dropped table
